# Pattern Matching Algorithm Speed Comparison in Jupyter Notebook

## 1. Introduction and problem description
In this project, we compare the performance of three string pattern-matching algorithms: 
1. Brute-force
2. Knuth-Morris-Pratt (KMP)
3. Boyer-Moore

These algorithms are fundamental in string searching tasks and are used in many applications including text editors, search engines, and DNA sequence analysis.
Our goal is to experimentally analyze their relative performance by measuring the time each takes to find patterns of various lengths in large text documents.

## 2. Description of the 3 algorithms and how they work

### I. Brute-force
Brute Force algorithm is a straightforward algorithmic approach that systematically explores all possible solutions to a problem until the correct one is found. It is often used when the problem space is small or when no optimized solution is available. This method is simple to implement but can be computationally expensive for larger problems due to its exhaustive nature.

#### Key Characteristics
Brute Force Search involves methodically testing every possible solution in a specified order. It does not use optimization techniques or heuristics, relying solely on exhaustive exploration. This makes it a generic approach applicable to various domains, such as string matching, combinatorial problems, and cybersecurity.

For example, in string search, the algorithm compares a substring against every possible position in the target string until a match is found. Similarly, in combinatorial problems, it tests all combinations of elements to find the desired result.


In [2]:
def brute_force_search(text, pattern):
    n = len(text)
    m = len(pattern)

    for i in range(n - m + 1): # Iterate through all possible starting positions
        match = True
        for j in range(m): # Compare each character of the pattern
            if text[i + j] != pattern[j]:
                match = False
                break
            if match:
                return i # Return the starting index of the match
    return -1 # Return -1 if no match is found


### II. Knuth-Morris-Pratt (KMP)
The Knuth-Morris-Pratt (KMP) algorithm is a string-searching algorithm that efficiently searches for occurrences of a "pattern" within a "text" by utilizing the information from previous matches to avoid unnecessary comparisons. This algorithm was developed by Donald Knuth, Vaughan Pratt, and James H. Morris.

#### Key Principles
The KMP algorithm preprocesses the pattern to create an auxiliary array called the Longest Prefix Suffix (LPS) array. This array helps in determining the next positions to match after a mismatch, thus avoiding redundant comparisons.

##### Preprocessing the Pattern
The LPS array is constructed to store the length of the longest proper prefix which is also a suffix for each sub-pattern of the given pattern. This preprocessing step ensures that the algorithm can skip certain comparisons during the search phase.

In [ ]:
def computeLPSArray(pattern):
    """
    Computes the Longest Prefix Suffix (LPS) array for the given pattern.
    The LPS array is used in the KMP algorithm to skip unnecessary comparisons.
    """
    M = len(pattern)
    lps = [0] * M  # Initialize LPS array
    length = 0     # Length of the previous longest prefix suffix
    i = 1
    while i < M:
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]  # Use previous LPS value
            else:
                lps[i] = 0
                i += 1
    return lps

##### Searching the Pattern

Once the LPS array is ready, the KMP algorithm uses it to search the pattern in the text. The algorithm compares characters of the pattern with the text and uses the LPS array to skip unnecessary comparisons when a mismatch occurs

In [ ]:
def KMPSearch(pattern, text):
    """
    Searches for occurrences of 'pat' in 'text' using the KMP algorithm.
    Returns a list of starting indices where 'pat' is found in 'text'.
    """
    M = len(pattern)
    N = len(text)
    lps = computeLPSArray(pattern)  # Preprocess the pattern to get the LPS array
    i = 0  # index for text
    j = 0  # index for pattern
    result = []
    while i < N:
        if pattern[j] == text[i]:
            i += 1
            j += 1
        if j == M:
            result.append(i - j)  # Pattern found, append starting index
            j = lps[j - 1]       # Continue to search for next possible match
        elif i < N and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]   # Use LPS to avoid unnecessary comparisons
            else:
                i += 1           # Move to the next character in text
    return result

### III. Boyer-Moore

The Boyer-Moore algorithm is a highly efficient string-searching algorithm, especially effective when the pattern is much shorter than the text. Unlike naive approaches, Boyer-Moore skips sections of the text, often resulting in faster searches.

#### Key Concepts

- **Right-to-Left Matching:** The algorithm compares the pattern to the text from right to left.
- **Bad Character Rule:** On a mismatch, the pattern is shifted so that the mismatched character in the text aligns with its last occurrence in the pattern. If the character is not present in the pattern, the pattern is shifted past the mismatched character.
- **Good Suffix Rule:** If a mismatch occurs after matching a suffix, the pattern is shifted to align the next occurrence of this suffix (or a matching prefix) in the pattern.

#### Steps of the Algorithm

1. Preprocess the pattern to create:
    - A bad character table.
    - A good suffix table.
2. Start comparing the pattern with the text from the rightmost character.
3. Use the heuristics to determine the shift after a mismatch.
4. Repeat until the pattern is found or the text is fully scanned.

In [5]:
def boyer_moore(text, pattern):
    # Preprocess the pattern for the bad character rule
    def preprocess_bad_character(pattern):
        bad_char_table = {}
        for i, char in enumerate(pattern):
            bad_char_table[char] = i  # Store the last occurrence of each character
        return bad_char_table

    # Preprocess the pattern for the good suffix rule
    def preprocess_good_suffix(pattern):
        m = len(pattern)
        good_suffix_table = [0] * m
        border_pos = [0] * (m + 1)
        i, j = m, m + 1
        border_pos[i] = j

        # Build border positions
        while i > 0:
            while j <= m and pattern[i - 1] != pattern[j - 1]:
                if good_suffix_table[j] == 0:
                    good_suffix_table[j] = j - i
                j = border_pos[j]
            i -= 1
            j -= 1
            border_pos[i] = j

        # Build good suffix table
        j = border_pos[0]
        for i in range(m + 1):
            if good_suffix_table[i] == 0:
                good_suffix_table[i] = j
            if i == j:
                j = border_pos[j]

        return good_suffix_table

    bad_char_table = preprocess_bad_character(pattern)
    good_suffix_table = preprocess_good_suffix(pattern)

    n, m = len(text), len(pattern)
    s = 0  # s is the shift of the pattern with respect to text

    while s <= n - m:
        j = m - 1
        # Compare pattern from right to left
        while j >= 0 and pattern[j] == text[s + j]:
            j -= 1
        if j < 0:
            print(f"Pattern found at index {s}")
            # Shift pattern according to good suffix rule
            s += good_suffix_table[0] if s + m < n else 1
        else:
            # Calculate shifts for bad character and good suffix rules
            bad_char_shift = j - bad_char_table.get(text[s + j], -1)
            good_suffix_shift = good_suffix_table[j + 1]
            s += max(bad_char_shift, good_suffix_shift)
